In [1]:
!git clone -b implementing-vars https://github.com/DrYogurt/Aero-Design-Team-Gamma.git
!pip install poetry==1.4.2

Cloning into 'Aero-Design-Team-Gamma'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 206 (delta 87), reused 155 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (206/206), 489.49 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (87/87), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of poetry-plugin-export to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter

In [1]:
%cd Aero-Design-Team-Gamma
!poetry config virtualenvs.in-project true
!poetry install --no-ansi

/content/Aero-Design-Team-Gamma
Creating virtualenv aerospace-design-team-gamma in /content/Aero-Design-Team-Gamma/.venv
Installing dependencies from lock file
The lock file might not be compatible with the current version of Poetry.
Upgrade Poetry to ensure the lock file is read properly or, alternatively, regenerate the lock file with the `poetry lock` command.

Package operations: 41 installs, 0 updates, 0 removals

  • Installing numpy (1.26.4)
  • Installing six (1.17.0)
  • Installing contourpy (1.3.1)
  • Installing cycler (0.12.1)
  • Installing fonttools (4.55.3)
  • Installing kiwisolver (1.4.8)
  • Installing packaging (24.2)
  • Installing pillow (11.1.0)
  • Installing pyparsing (3.2.1)
  • Installing python-dateutil (2.9.0.post0)
  • Installing click (8.1.8)
  • Installing iniconfig (2.0.0)
  • Installing mccabe (0.7.0)
  • Installing matplotlib (3.10.0)
  • Installing markupsafe (3.0.2)
  • Installing mpmath (1.3.0)
  • Installing mypy-extensions (1.0.0)
  • Installing p

In [2]:
VENV_PATH = "/content/Aero-Design-Team-Gamma/.venv/lib/python3.11/site-packages"
import os, sys
LOCAL_VENV_PATH = '/content/venv' # local notebook
os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
sys.path.insert(0, LOCAL_VENV_PATH)

In [3]:
from ambiance import Atmosphere

from global_variables.solver import EquationSystem
from global_variables.registry import VariableRegistry, Variable

registry = VariableRegistry("aero_vars.yaml")

In [4]:
outputs = {"S_TO", "R", "sigma_max"}

# Create solvers for each output
solvers = {
    output: EquationSystem(registry, {output})
    for output in outputs
}

# Get all unique inputs
all_inputs = set().union(*(solver.inputs for solver in solvers.values()))
print(f"Required inputs: {all_inputs}")
solvers

Required inputs: {'V', 'C_Lmax', 'W_pax', 'S_', 'We_Wmax', 'T_A0', 'e', 'mu', 'rho', 'T_s', 'W_max', 'TSFC', 'n_pax', 'b', 'C_D0'}


{'R': <global_variables.solver.EquationSystem at 0x7af92bbf7990>,
 'sigma_max': <global_variables.solver.EquationSystem at 0x7af8f92bc590>,
 'S_TO': <global_variables.solver.EquationSystem at 0x7af92bbe5550>}

In [5]:
def mach(M,h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.speed_of_sound[-1] * M / 0.3048

In [6]:
def rho_func(h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.density[-1]

In [7]:

# Generate sample points for each input
n_samples = 1000
samples = {}

# Define ranges for each input (you'll need to adjust these)
ranges = {
    'V':(mach(0.8,35e3),mach(0.95,35e3)), #cruising velocity ft / s
    'S_': (8e4,12e4), # area in ft^2
    'b': (200, 400),  # wing area in ft^2
    'T_A0': (4e5,6e5),
    'W_pax': (185,300), #weight per passenger in lbs
    'e':(0.8,1.), # oswald efficiency
    'mu':(.01,.05),
    'W_max':(1.75e6, 2.25e6),  # weight in lbs
    'C_Lmax':(1.5,2.),  # weight in lbs
    'C_D0':(0.01,0.35),
    'rho':(rho_func(45e3),rho_func(0)),
    'n_pax': (1200,1260), # number of passengers
    'We_Wmax':(0.4,0.6),
    #'h': (0, 40000),  # altitude in ft
    'TSFC': (0.3,0.6), # TSFC

    # Add other inputs and their ranges
}
print(ranges)

{'V': (778.514759070209, 924.4862763958732), 'S_': (80000.0, 120000.0), 'b': (200, 400), 'T_A0': (400000.0, 600000.0), 'W_pax': (185, 300), 'e': (0.8, 1.0), 'mu': (0.01, 0.05), 'W_max': (1750000.0, 2250000.0), 'C_Lmax': (1.5, 2.0), 'C_D0': (0.01, 0.35), 'rho': (0.23824523567640446, 1.225000018124288), 'n_pax': (1200, 1260), 'We_Wmax': (0.4, 0.6), 'TSFC': (0.3, 0.6)}


In [8]:
import numpy as np
import corner
import matplotlib.pyplot as plt
# Generate Latin Hypercube samples
from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=len(all_inputs))
sample_points = sampler.random(n=n_samples)

# Scale samples to actual ranges
input_list = sorted(all_inputs)
for i, input_var in enumerate(input_list):
    if input_var in ranges:
        min_val, max_val = ranges[input_var]
        samples[input_var] = qmc.scale(sample_points[:, [i]], [min_val], [max_val]).flatten()
    else:
        print(f"Warning: No range defined for {input_var}")
        samples[input_var] = sample_points[:, i]  # Use 0-1 range as default

In [ ]:
# Calculate outputs
results = {}
for output, solver in solvers.items():
    solver_func = solver.create_solver()[output]
    input_args = [samples[inp] for inp in sorted(solver.inputs)]
    try:
        output_values = np.array(solver_func(*input_args))
        print(output_values)
        # Ensure output is 1D array of correct length
        if output_values.size == 1:
            output_values = np.full(n_samples, output_values)
        results[output] = output_values
    except Exception as e:
        print(f"Error calculating {output}: {e}")
#results

In [ ]:
# Create corner plot for each output
for output, output_values in results.items():
    solver = solvers[output]
    # Combine input and output data
    input_arrays = [samples[inp] for inp in sorted(solver.inputs)]
    data = np.column_stack(input_arrays + [output_values])
    labels = list(sorted(solver.inputs)) + [output]

    # Create corner plot
    fig = corner.corner(
        data,
        labels=labels,
        #show_titles=True,
        #title_kwargs={"fontsize": 12},
        #plot_datapoints=True,
        #plot_density=True,
        #plot_contours=True,
        #fill_contours=True,
        #smooth=1.0
    )

    fig.suptitle(f"Corner Plot for {output}", fontsize=14, y=1.02)
    plt.savefig(f"corner_plot_{output}.png", bbox_inches='tight', dpi=300)
    plt.close()

In [19]:
R_func, soln = solvers['R'].create_solver()
R_func['R'](0,0,0,0,0,0,0,0,0,0,0)

R

In [12]:
solvers['R']._collect_equations()

{'C_L_full': ['2 * W_max / (rho * V**2 * S_)'],
 'CL_CD_ratio': ['0.5 * ((C_L_full)**(1/2)/C_D_full + (C_L_empty)**(1/2)/C_D_empty)'],
 'W_e': ['We_Wmax * W_max'],
 'C_L_empty': ['2 * (W_max - W_f) / (rho * V**2 * S_)'],
 'AR': ['b**2 / S_'],
 'C_D_empty': ['C_D0 + C_Di_empty'],
 'W_p': ['n_pax * W_pax'],
 'C_D_full': ['C_D0 + C_Di_full'],
 'C_Di_empty': ['C_L_empty**2 / (pi * e * AR)'],
 'R': ['(2/TSFC) * (2/(rho*S_)**(1/2)) * CL_CD_ratio * ((W_max)**(1/2)-(W_max-W_f)**(1/2))'],
 'C_Di_full': ['C_L_full**2 / (pi * e * AR)'],
 'W_f': ['W_max - W_p - W_e']}